Aware Super data cleaning template  

In [1]:
import pandas as pd

df = pd.read_excel('aware.xlsx', sheet_name = 'Table1')

# Sneak peak of the data
print(df.info())
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2617 entries, 0 to 2616
Data columns (total 13 columns):
 #   Column                                                                                                               Non-Null Count  Dtype 
---  ------                                                                                                               --------------  ----- 
 0   PHD SCHEDULE 8D TABLE 1 - PORTFOLIO HOLDINGS INFORMATION FOR INVESTMENT OPTION [AWAREACCGROW] - ASSETS - 2024-12-31  2616 non-null   object
 1   Unnamed: 1                                                                                                           2598 non-null   object
 2   Unnamed: 2                                                                                                           2598 non-null   object
 3   Unnamed: 3                                                                                                           2598 non-null   object
 4   Unnamed: 4        